### Tests for Applying Gains Matrix

In [8]:
import cupy as cp
import numpy as np
from utils import *
from zp_puregpu_funcs_py import *

In [9]:
apply_gains_to_mat?

Signature:
apply_gains_to_mat(
    gains,
    mat,
    edges,
    ant_1_array,
    ant_2_array,
    xp,
    is_zeropadded=True,
)
Docstring:
Apply a pair of complex gains to a matrix. Utilizes the Re/Im split.
Only accounts for "one half" of the gain application, meaning the 
function is really performing (g_1g_2*\Delta_{1,2}), where it is 
understood that antenna's 1 and 2 below to the baseline sitting at
the same row as that baseline row in the \Delta matrix.

NOTE: Could be smart in the future to wrap the zeropadding of the gain 
    matrix in a separate function.

Params
------
mat: Gains are applied to this. Can be 2d as in original C-corrcal.
    If 3d, is_zeropadded must be set to True

Returns
-------
out: Matrix with applied gains (explain this a bit better)
File:      ~/corrcal_gpu_pipeline/pipeline/utils.py
Type:      function

In [54]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 10
n_bl = 2*n_ant
n_gains = 4*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = 2)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_ant)
ant_2_array = cp.arange(n_ant, 2*n_ant)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

The edges of the redundant blocks have indices[ 0  6 16 20]


In [57]:
applied_gains = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_1_array, cp, True)

In [58]:
print(f"Input mat has shape {sim_diff_mat_3d.shape}")
print()
print(f"Gains applied mat has shape {applied_gains.shape}")
print()
print(f"The mat with gains applied looks like:\n\n{applied_gains}")

Input mat has shape (3, 10, 3)

Gains applied mat has shape (3, 10, 3)

The mat with gains applied looks like:

[[[3.47591769e-01 4.80654363e-01 3.88563391e-01]
  [2.58249775e-01 1.75625572e-01 3.03918038e-01]
  [1.95836328e-03 1.25368739e-03 1.38064132e-03]
  [1.07107959e-03 4.37554910e-03 9.79744614e-04]
  [4.01285451e-03 1.11756963e-02 1.14032073e-02]
  [2.25805388e-01 4.88176633e-02 1.39965524e-01]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[2.44146951e-02 4.36355669e-02 2.03987858e-02]
  [4.73174599e-02 4.38763468e-02 1.60734251e-03]
  [8.16648918e-01 2.59986637e-01 3.82632711e-01]
  [8.75108143e-01 9.98599759e-01 1.04949981e+00]
  [2.18890259e-01 1.41107322e-01 3.38017245e-01]
  [4.44954806e-01 4.68779237e-01 1.08312055e-01]
  [8.01467967e-01 4.01624667e-01 2.53512877e-01]
  [7.95029736e-01 7.44491879e-01 7.55623402e-01]
  [4

#### Random Debugging tests

In [ ]:
m = np.random.rand(2, 5, 3)
g = np.random.rand(10)

gs = g[:, None] * g[:, None]
gs = gs.reshape(m.shape[0], m.shape[1], 1)

print(gs.shape)
print(m.shape)
print(g.shape)
print(m)
print(m[:, ::2])
out = gs[:, ::2] * m[:, ::2]

In [ ]:
a = np.arange(5)
b = np.arange(5,10)
c = np.random.rand((a.shape[0] + b.shape[0]))
print(c)
d = c[a]
e = c[b]

print(d)
print()
print(e)
print()
print(np.concatenate((d, e)))
print()
aa = np.concatenate((d, e))

print(aa[:int(len(aa)/2)])

print(aa[int(len(aa)/2):])